In [5]:
import os

import pandas as pd

from dotenv import dotenv_values
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv

POSTGRES_USER='dbtuser'
POSTGRES_PASSWORD='pssd'
POSTGRES_HOST='localhost'
POSTGRES_PORT=5433
POSTGRES_DB='dbtdb'


CONFIG = dotenv_values('.env')
if not CONFIG:
    CONFIG = os.environ

connection_uri = "postgresql://{}:{}@{}:{}/{}".format(
   POSTGRES_USER,
   POSTGRES_PASSWORD,
    POSTGRES_HOST,
   POSTGRES_PORT,
   POSTGRES_DB,
)

In [8]:
engine = create_engine(connection_uri, pool_pre_ping=True)
engine.connect()

In [10]:


data = 'D:/tenacademy/dbt_airflow/datafiles/data.csv'

with open(data, 'r') as file:
    lines = file.readlines()

# df = pd.read_csv(data,index_col=False, delimiter='; ')
# df.drop_duplicates(inplace=True)
# df.head(10)

In [11]:
print(f"The number of rows/lines is {len(lines)}")

The number of rows/lines is 923


In [12]:
print(lines[0]) # column names
print(lines[0].strip('\n').strip().strip(';').split(';')) # columns names as a list

track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time

['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [13]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
len(lines_as_lists)

923

In [ ]:
print(f"the number of fields in row 1 is {len(lines_as_lists[1])}, row 2 is {len(lines_as_lists[2])}")

In [14]:
no_field_max = 0

for row in lines_as_lists:
    if len(row) > no_field_max:
        no_field_max = len(row)

print(f"the maximum number of fields is {no_field_max}")
largest_n = int((no_field_max-4)/6)
print(f"the largest n = {largest_n}")

the maximum number of fields is 122830
the largest n = 20471


In [15]:
cols = lines_as_lists.pop(0)

In [16]:
track_cols = cols[:4]
trajectory_cols = ['track_id'] + cols[4:]

print(track_cols)
print(trajectory_cols)

['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [17]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4]) 

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix

In [18]:
df_trafic1 = pd.DataFrame(data= track_info,columns=track_cols)
df_trafic1.head(10)

,track_id,type,traveled_d,avg_speed
0,1,Car,48.85,9.770344
1,2,Motorcycle,98.09,19.839417
2,3,Motorcycle,63.80,18.228752
3,4,Motorcycle,145.72,26.229014
4,5,Motorcycle,138.01,24.841425
5,6,Medium Vehicle,159.97,17.451502
6,7,Motorcycle,153.91,22.895895
7,8,Car,158.21,21.739084
8,9,Car,161.01,21.154471
9,10,Car,159.53,18.891989


In [19]:
df_trafic2 = pd.DataFrame(data= trajectory_info,columns=trajectory_cols)
df_trafic2.head(10)


,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.977391,23.737688,4.9178,0.0518,-0.0299,0.000000
1,1,37.977391,23.737688,4.9207,-0.0124,-0.0354,0.040000
2,1,37.977391,23.737688,4.9160,-0.0519,-0.0413,0.080000
3,1,37.977390,23.737688,4.9057,-0.0914,-0.0478,0.120000
4,1,37.977390,23.737689,4.8871,-0.1679,-0.0550,0.160000
5,1,37.977389,23.737689,4.8547,-0.2815,-0.0628,0.200000
6,1,37.977389,23.737689,4.8033,-0.4320,-0.0713,0.240000
7,1,37.977388,23.737690,4.7318,-0.5617,-0.0804,0.280000
8,1,37.977388,23.737690,4.6442,-0.6543,-0.0905,0.320000
9,1,37.977388,23.737690,4.5437,-0.7413,-0.1017,0.360000


In [20]:
postgrsql = df_trafic1.to_sql('df_trafic1', engine, index=False, if_exists='replace')
engine.dispose()

postgrsql = df_trafic2.to_sql('df_trafic2', engine, index=False, if_exists='replace')
engine.dispose()


In [21]:

pd.read_sql(f"SELECT * FROM df_trafic1", engine) 


,track_id,type,traveled_d,avg_speed
0,1,Car,48.85,9.770344
1,2,Motorcycle,98.09,19.839417
2,3,Motorcycle,63.80,18.228752
3,4,Motorcycle,145.72,26.229014
4,5,Motorcycle,138.01,24.841425
...,...,...,...,...
917,918,Car,78.83,30.846243
918,919,Motorcycle,19.50,9.234518
919,920,Car,48.97,24.486209
920,921,Motorcycle,46.68,30.007124


In [22]:
pd.read_sql(f"SELECT * FROM df_trafic2", engine) 

,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.977391,23.737688,4.9178,0.0518,-0.0299,0.000000
1,1,37.977391,23.737688,4.9207,-0.0124,-0.0354,0.040000
2,1,37.977391,23.737688,4.9160,-0.0519,-0.0413,0.080000
3,1,37.977390,23.737688,4.9057,-0.0914,-0.0478,0.120000
4,1,37.977390,23.737689,4.8871,-0.1679,-0.0550,0.160000
...,...,...,...,...,...,...,...
1446882,922,37.978596,23.735866,21.3406,-0.1442,-0.0672,818.640000
1446883,922,37.978597,23.735868,21.3323,-0.1494,-0.0637,818.680000
1446884,922,37.978598,23.735870,21.3226,-0.1679,-0.0601,818.720000
1446885,922,37.978599,23.735872,21.3095,-0.1968,-0.0564,818.760000
